In [1]:
sc

In [17]:
data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('coalesce_joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)


df.registerTempTable('temp')

In [45]:
# df = df.filter(df.LATITUDE != '0')

# test = df.filter(df.DATETIME == '2016-06-20 21:06:00')
# test = df.filter(if '2016-06-20 21' in df.DATETIME)

# test.count()
df.first()

# a.coalesce(1).write.csv('geodata', header=True)

Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=u'63.0', Events=None, Fre

In [24]:
year = "2016%"
query = 'select * from temp where DATETIME LIKE "{0}"'.format(year)
temp = sqlContext.sql(query)

In [37]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def filter_year(x):
    return x.split("-")[0]
    
year_filter = udf(filter_year, StringType())
newcol_df = df.withColumn("YEAR", year_filter("DATETIME"))
unique_years = newcol_df.select("YEAR").distinct()
unique_years_list = unique_years.rdd.flatMap(lambda x: x).collect()

# unique_years = df.select()

In [38]:
unique_years_list

[u'2016', u'2017']

In [43]:
for year in unique_years_list:
    year_wildcard = year+"%"
    query = 'select * from temp where DATETIME LIKE "{0}"'.format(year_wildcard)
    temp = sqlContext.sql(query)
#     print(temp.take(1))
    temp.coalesce(1).write.csv('partitioned_years/joins_in_'+year, header=True)